In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
import gensim
import sklearn
import jsonlines
import re
import os
from datetime import datetime

import json
import pandas as pd


In [2]:
def flatten_dict(pyobj, keystring=''): 
    if type(pyobj) == dict: 
        keystring = keystring + '_' if keystring else keystring 
        for k in pyobj: 
            yield from flatten_dict(pyobj[k], keystring + str(k)) 
    else: 
        yield keystring, pyobj 



In [3]:
def get_hashtags(hash_list):
    hashtags = ''
    for item in hash_list:
        hashtags = hashtags + ' ' + item['text']
        hashtags = hashtags.strip()
    return hashtags

In [4]:
key_names = ['lang', 'quoted_status_lang', 'created_at', 'id_str', 'full_text', 
             'entities_hashtags', 'entities_urls', 'entities_media',
             'quoted_status_full_text']

In [5]:
my_file = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-00.jsonl'
my_file2 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-01.jsonl'
my_file3 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-02.jsonl'
my_file4 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-03.jsonl'
my_file5 = '/Volumes/LACIE_SHARE/python_scripts/Data/coronavirus-tweet-id-2020-02-01-09.jsonl'

In [6]:
data = []



for file in [my_file, my_file2]:
    with jsonlines.open(my_file) as f:
        for line in f:
            # keep only if english and not a retweet
            if line['lang'] == 'en':
                data.append(dict(flatten_dict(line)))
    print('Loaded {} records'.format(len(data)))

Loaded 47409 records
Loaded 94818 records


In [34]:
text_only = []
for d in data:
    text_only.append(d.get('full_text',''))

#We should eventually remove URLs and the username of someone if we are retweeting.
text_only_df = pd.DataFrame(text_only, columns = ['text'])

In [62]:
text_only_df = text_only_df.sample(n = 5000)

In [63]:
#text_only_df.info

<bound method DataFrame.info of                                                     text  \
8429   RT @DanyalGilani: Salute to these Chinese doct...   
46184  RT @SethAbramson: If you've read *anything* ab...   
18071  RT @Huh_My_Rahhhhh: Father can’t touch his lit...   
90076  RT @pautergeist: population to this highly con...   
49317  RT @AJListeningPost: The coronavirus outbreak ...   
...                                                  ...   
67607  RT @Marfoogle: IF THIS IS REAL? then the futur...   
7229   RT @popplioikawa: the flu: *results in 500,000...   
81723  RT @catcontentonly: Your racist jokes about dy...   
22329  RT @mercola: Read my thoughts on the Coronavir...   
73459  RT @Pdiddy1805: @SteveKerr You sure do have a ...   

                                                  tokens  \
8429   [salute, chinese, doctor, working, tirelessly,...   
46184  [read, anything, trump, activity, china, turke...   
18071  [father, touch, little, baby, infected, corona...   
90076  

In [64]:
## Check for and remove missing values and blank strings
print(text_only_df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 8429 to 73459
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   tokens  5000 non-null   object
 2   clean   5000 non-null   object
dtypes: object(3)
memory usage: 156.2+ KB
None


In [65]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


sw = set(stopwords.words('english'))
sw.add('rt')
wn = WordNetLemmatizer()

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


def tokenize(x):
    x = re.sub(r'http\S+', '', x) ## remove any urls in the tweets
    x = re.sub(r'@\S+','', x) #remove any usernames in the tweets
    x = emoji_pattern.sub(r'', x)
    
    x = x.lower()
    x = x.replace('\r', '') ## REmove \r \n
    x = x.replace('\n', '') #remove \n
    x = x.replace("\'", "'") #replace \' with '
    x = x.replace("#", "") ## Remove hashtags
    #x = x.replace("@", "") ## remove the @ symbol for retweets
    tokens = wordpunct_tokenize(x)
    tokens = [tok for tok in tokens if tok.isalnum()]
    
    tokens = [tok for tok in tokens if tok not in sw]
    
    tokens = [wn.lemmatize(tok) for tok in tokens]
    return(tokens)



text_only_df['tokens'] = text_only_df['text'].apply(tokenize)
text_only_df['clean'] = text_only_df['tokens'].apply(lambda x: " ".join(x))

In [70]:
text_only_df['text']

8429     RT @DanyalGilani: Salute to these Chinese doct...
46184    RT @SethAbramson: If you've read *anything* ab...
18071    RT @Huh_My_Rahhhhh: Father can’t touch his lit...
90076    RT @pautergeist: population to this highly con...
49317    RT @AJListeningPost: The coronavirus outbreak ...
                               ...                        
67607    RT @Marfoogle: IF THIS IS REAL? then the futur...
7229     RT @popplioikawa: the flu: *results in 500,000...
81723    RT @catcontentonly: Your racist jokes about dy...
22329    RT @mercola: Read my thoughts on the Coronavir...
73459    RT @Pdiddy1805: @SteveKerr You sure do have a ...
Name: text, Length: 5000, dtype: object

In [67]:
## Run LDA topic modeling on the tweet text



import gensim.corpora as corpora

ntopics = 10

# Create Dictionary
id2word = corpora.Dictionary(text_only_df['tokens'])

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in text_only_df['tokens']]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=ntopics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)


ldatopics = lda_model.show_topics(formatted=False)
print(lda_model.print_topics())

[(0, '0.133*"coronavirus" + 0.039*"case" + 0.037*"u" + 0.026*"new" + 0.020*"health" + 0.016*"first" + 0.013*"amp" + 0.013*"update" + 0.013*"confirmed" + 0.013*"say"'), (1, '0.037*"one" + 0.025*"mask" + 0.024*"come" + 0.023*"mean" + 0.018*"well" + 0.016*"foreign" + 0.014*"information" + 0.014*"market" + 0.013*"hedge" + 0.013*"still"'), (2, '0.000*"pc" + 0.000*"ya" + 0.000*"gimme" + 0.000*"todragkoreanartistsstopusingcoronavirus" + 0.000*"peed" + 0.000*"poop" + 0.000*"knowing" + 0.000*"stopusingcoronavirus" + 0.000*"egg" + 0.000*"log"'), (3, '0.062*"coronavirus" + 0.054*"wuhan" + 0.035*"chinese" + 0.024*"outbreak" + 0.021*"state" + 0.018*"man" + 0.018*"report" + 0.015*"time" + 0.014*"united" + 0.013*"said"'), (4, '0.000*"pc" + 0.000*"ya" + 0.000*"gimme" + 0.000*"todragkoreanartistsstopusingcoronavirus" + 0.000*"peed" + 0.000*"poop" + 0.000*"knowing" + 0.000*"stopusingcoronavirus" + 0.000*"egg" + 0.000*"log"'), (5, '0.060*"000" + 0.046*"china" + 0.036*"trump" + 0.033*"death" + 0.032*"flu"

In [69]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
0     -0.239039+0.000000j  0.282500+0.000000j       1        1  26.768891
5     -0.303535+0.000000j -0.263612+0.000000j       2        1  18.169238
3     -0.104966+0.000000j  0.212816+0.000000j       3        1  16.903101
7     -0.077673+0.000000j -0.187912+0.000000j       4        1  15.673923
6      0.110952+0.000000j -0.017975+0.000000j       5        1   9.515220
1      0.108597+0.000000j -0.028831+0.000000j       6        1   9.044521
4      0.126416+0.000000j  0.000753+0.000000j       7        1   0.981277
8      0.126416+0.000000j  0.000753+0.000000j       8        1   0.981277
9      0.126416+0.000000j  0.000753+0.000000j       9        1   0.981277
2      0.126416+0.000000j  0.000753+0.000000j      10        1   0.981277, topic_info=             Term         Freq        Total Category  logprob  loglif

In [ ]:
##TODO: remove retweets?
